<a href="https://colab.research.google.com/github/TSGreenwood/tsgreenwood.github.io/blob/google_cloud/map_bike_lanes_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Map bike lanes using Folium**

In [1]:
# Set the current project in Google Cloud SDK to 'taragreenwoodpersonalwebsite'

!gcloud config set project 'taragreenwoodpersonalwebsite'

Updated property [core/project].


In [2]:
# Connect notebook to my Drive account

from google.colab import auth
auth.authenticate_user()

In [3]:
# Navigate to my repo

!git clone https://github.com/TSGreenwood/tsgreenwood.github.io.git

Cloning into 'tsgreenwood.github.io'...
remote: Enumerating objects: 1363, done.
remote: Counting objects: 100% (1363/1363), done.
remote: Compressing objects: 100% (578/578), done.
remote: Total 1363 (delta 834), reused 1253 (delta 759), pack-reused 0
Receiving objects: 100% (1363/1363), 37.46 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (834/834), done.


In [4]:
%cd tsgreenwood.github.io/

/content/tsgreenwood.github.io


In [ ]:
#! ls

 CNAME		  index.html				    menu.html
 css		  js					   'python_foundation_environment .yml'
 embedding.html   king_county_public_transit_options.html   README.md
 empty.html	  kingcountytransitmapping		    resume.html
 footer.html	  LICENSE				    static
 img		  mapbox_gl_js_tutorial.html		    widgets.html


In [5]:
# Switch branches

!git checkout google_cloud

Branch 'google_cloud' set up to track remote branch 'google_cloud' from 'origin'.
Switched to a new branch 'google_cloud'


In [6]:
# Pull the latest changes

!git pull origin google_cloud

From https://github.com/TSGreenwood/tsgreenwood.github.io
 * branch            google_cloud -> FETCH_HEAD
Already up to date.


In [ ]:
# If config problems
#!git config user.email "greenwood.ts@gmail.com"
#!git config user.name "Greenwood"

In [ ]:
#!git merge origin/google_cloud --no-edit

Merge made by the 'ort' strategy.
 connect_github_cloud_colab_template.ipynb                       | 497 +++++++++++++++++++++++++++
 .../python_notebooks/bike_routes_preprocessing.ipynb            | 298 ++++++++++++++++
 .../python_notebooks/cloud_colab_notebook_template.ipynb        |  93 +++++
 .../python_notebooks/map_bike_lanes_folium.ipynb                | 470 ++++++++++++++++++++++++++
 .../python_notebooks/mapping_bike_lanes_folium.ipynb            |   0
 kingcountytransitmapping/utils/generalized_mapping_functions    |  27 ++
 kingcountytransitmapping/utils/mapping_functions.py             |  71 ++++
 map_bike_lanes_folium.ipynb                                     | 505 ++++++++++++++++++++++++++++
 map_template_folium.ipynb                                       | 387 +++++++++++++++++++++
 9 files changed, 2348 insertions(+)
 create mode 100644 connect_github_cloud_colab_template.ipynb
 create mode 100644 kingcountytransitmapping/python_notebooks/bike_routes_preprocessing.ipy

In [7]:
# Import my mapping functions from the mapping_functions.py file located in my repository into Colab notebook

!wget https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py

--2024-03-25 03:27:30--  https://raw.githubusercontent.com/TSGreenwood/tsgreenwood.github.io/google_cloud/kingcountytransitmapping/utils/mapping_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829 (2.8K) [text/plain]
Saving to: ‘mapping_functions.py’

mapping_functions.p 100%[===================>]   2.76K  --.-KB/s    in 0s      

2024-03-25 03:27:30 (46.3 MB/s) - ‘mapping_functions.py’ saved [2829/2829]



In [8]:
import mapping_functions

In [ ]:
! git status

On branch king_county_mapbox_tiles
Your branch is ahead of 'origin/king_county_mapbox_tiles' by 28 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/
	mapping_functions.py

nothing added to commit but untracked files present (use "git add" to track)


In [23]:
# Initialize basemap

m1 = mapping_functions.create_anchored_kc_map()

AttributeError: module 'mapping_functions' has no attribute 'create_anchored_kc_map'

In [ ]:
# Work directly with GeoJSON dataset stored in Google Cloud bucket

import json
from google.cloud import storage

In [ ]:
# Access the GeoJSON File from the Bucket

# Initialize a client
storage_client = storage.Client()

# Define your bucket name and file name
bucket_name = 'kingcountypublictransit'
blob_name = 'data/bike_lanes.geojson'

# Create a bucket object and a blob (file) object
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)

In [ ]:
# Ddownload the GeoJSON file directly into memory and load it as a JSON object. This approach avoids saving the file locally

# Download the file's content into memory
geojson_as_string = blob.download_as_string()

# Convert the string to a JSON object
geojson_data = json.loads(geojson_as_string)

Now that the dataset is loaded, we'll do some mapping.

In [ ]:
m1

In [24]:
import folium

def create_anchored_kc_base_map(default_location=[47.60805584750898, -122.30272433294837],
                                width='300px', height='485.4px', default_zoom_start=10):
    m = folium.Map(location=default_location, zoom_start=default_zoom_start,
                   tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                   attr='Esri', overlay=False, control_scale=True, min_zoom=8)

    # Define your boundary for King County or the specific area of interest
    boundary = {
        'north': 47.734145,
        'south': 47.491912,
        'east': -121.8,
        'west': -122.4
    }

    # Apply the max bounds to the map
    m.fit_bounds([
        [boundary['south'], boundary['west']],
        [boundary['north'], boundary['east']]
    ])

    # Ensure user cannot pan outside of the given boundary
    m.options['maxBounds'] = [
        [boundary['south'], boundary['west']],
        [boundary['north'], boundary['east']]
    ]

    return m


In [25]:
m2 = create_anchored_kc_base_map()

In [13]:
import folium
from folium import plugins

In [ ]:
m2